# Image Scaling with Machine Learning

## Parameters to generate a set of images for train-test-validate
    * Over all parameters
        * number of image sets
        * sizes dict {name1: [h, w], name2: [h, w],...}
        * run-limits {escape_boundry: diagonal_multiplier, max_iterations: int}
        * coloring scheme - perhaps a function
        * output file format
        * ouput file naming format
    * File system
        * output directory {train: path, test: path, validate: path}
        * run-log location
    * Stocastic Choices:
        * hashed dictionary - to reject duplicate images
        * equation
            * parameters
        * rotation 
        * center point 
        * frame scale

## Source structure
    * main - opens run_parameters file - passes to module function
    * selection and tracking module
    * starting plain complex
    * equations module
    * iteration module
    * matrix to image module
[PIL image](https://pillow.readthedocs.io/en/5.1.x/reference/Image.html#the-image-class) <br>
****
[GitHub scalygraphic](https://github.com/dlanier/scalygraphic/) <br>
### Background Code:
[Add: random plane on domain](https://github.com/dlanier/scalygraphic/blob/master/scalygraphic/zplain.py) <br>

#### Import & Expand *functions_demo.py*
[FMF functions_demo](https://github.com/dlanier/FlyingMachineFractal/blob/master/pyreimpic/functions_demo_01.py) <br>
#### Import & Trim itergators.py
[FMF itergators](https://github.com/dlanier/FlyingMachineFractal/blob/master/src/itergataters.py) <br>
#### Rewrite HSV coloring
[FMF graphic_utility.py](https://github.com/dlanier/FlyingMachineFractal/blob/master/src/graphic_utility.py) <br>
[FMF numcolorpy.py](https://github.com/dlanier/FlyingMachineFractal/blob/master/src/numcolorpy.py) <br>
#### Reslove plain functions
[FMF z_plane.py](https://github.com/dlanier/FlyingMachineFractal/blob/master/src/z_plane.py) <br>
****
    
### Inception: Random Selection, Tracking and Generation - called from *main()*

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Clickety Clack:
## production_parameters: *output_dir_name, image_sizes_dict, number_of_sets*
### get a random eq & p with a random domain
#### check uniquness
### get a random color map
## Iterate: run, write, report

In [ ]:
#    ../src/im_scale_products.py
import os
import sys
import time
import hashlib
import inspect
import numpy as np
import PIL

sys.path.insert(0, '../src/')

import zplain as zp
import eq_iter
import deg_0_ddeq
import numcolorpy as ncp

"""     Constants (lookups):

        EQUS_DICT is an enumerated dictionary of the functions in module deg_0_ddeq
        EQUS_DICT_NAMED_IDX is a dictionary - name: number index of EQUS_DICT 
"""
EQUS_DICT = {k: v for k, v in enumerate(inspect.getmembers(deg_0_ddeq, inspect.isfunction))}
EQUS_DICT_NAMED_IDX = {v[0]: k for k, v in EQUS_DICT.items()}

def get_rand_eq_p_set():
    """ get a random equation and parameter set from the deg_0_ddeq module
    (No Args:)
    Returns:
        tuple:      (function_name, function_handle, parameter_set)
    """
    n = np.random.randint(0,len(EQUS_DICT),1)
    fcn_name, fcn = EQUS_DICT[n[0]]
    p = fcn(0.0, None)

    return (fcn_name, fcn, p)

def get_eq_by_name(fcn_name):
    """ Usage: function_handle = get_eq_by_name(fcn_name)
    Args:
        name of a function in deg_0_ddeq
    """
    if fcn_name in EQUS_DICT_NAMED_IDX:
        return EQUS_DICT[EQUS_DICT_NAMED_IDX[fcn_name]][1]
    else:
        return None
    
def get_random_domain(bounds_dict=None):
    """ Usage: 
    domain_dict = get_random_domain(h, w, bounds_dict)
    
    Args:
        bounds_dict:    min - max limits for keys eg.
                            CP_magnitude_limits = {'min': 0, 'max': 7}
                            ZM_limits           = {'min': np.finfo(float).eps, 'max': 2}
                            theta_limits        = {'min': 0, 'max':2 * np.pi}
                        
    Returns:
        domain_dict:    with keys:
                            center_point
                            zoom
                            theta
    """
    domain_dict = {}
    if bounds_dict is None:
        CP_magnitude_limits =   {'min': 0, 'max': 2}
        ZM_limits =             {'min': np.finfo(float).eps, 'max': 1}
        theta_limits =          {'min': 0, 'max':2 * np.pi}
    else:
        CP_magnitude_limits =   bounds_dict['CP_magnitude_limits']
        ZM_limits =             bounds_dict['ZM_limits']
        theta_limits =          bounds_dict['theta_limits']

    r = np.random.uniform(low=0.0, high=2*np.pi) * 0.0+1.0j
    m = np.random.uniform(low=CP_magnitude_limits['min'], high=CP_magnitude_limits['max'])
    domain_dict['center_point'] = m*np.exp(r)
    domain_dict['zoom'] = np.random.uniform(low=ZM_limits['min'], high=ZM_limits['max'])
    domain_dict['theta'] = np.random.uniform(low=theta_limits['min'], high=theta_limits['max'])
    
    return domain_dict

def sha256sum(s):
    """ Usage:  hash_key = sha256sum(input_str)
    """
    h  = hashlib.sha256()
    h.update(bytes(s, 'ascii'))
    
    return h.hexdigest()

def hash_parameters(domain_dict, fcn_name, p):
    """ Usage: hash_key = hash_parameters(domain_dict, fcn_name, p)
    """
    N_DEC = 15
    f = zp.get_frame_from_dict(domain_dict)
    s = zp.complex_frame_dict_to_string(f, N_DEC) + '\n' + fcn_name
    if isinstance(p, list):
        p_str = ''
        for p_n in p:
            p_str += zp.complex_to_string(p_n, N_DEC)
    else:
        p_str = zp.complex_to_string(p, N_DEC)
    
    s += p_str
    
    return sha256sum(s)

def get_im(ET, Z, Z0):
    """ Usage: I = get_im(ET, Z, Z0)
    """
    Zd, Zr, ETn = ncp.etg_norm(Z0, Z, ET)

    A = np.zeros((domain_dict['n_rows'],domain_dict['n_cols'],3))
    A[:,:,0] += ETn     # Hue
    A[:,:,1] += Zr      # Saturation
    A[:,:,2] += Zd      # Value
    I = PIL.Image.fromarray(np.uint8(A * 255), 'HSV').convert('RGB')
    
    return I

def now_name(prefi_str=None, suffi_str=None):
    """ get a human readable time stamp name """
    t0 = time.time()
    t_dec = t0 - np.floor(t0)
    ahora_nombre = time.strftime("%a_%d_%b_%Y_%H_%M_%S", time.localtime()) 
    if prefi_str is None: prefi_str = ''
    if suffi_str is None: suffi_str = ''
        
    return prefi_str + '_' + ahora_nombre + suffi_str


## get a random function and parameter set

In [ ]:
n_trys = 3

for k in range(n_trys):
    fcn_name, eq, p = get_rand_eq_p_set()
    
    print('\n\t', fcn_name)
    try:
        Z = 0.0+0.0j
        print(Z)
        print(eq(Z,p))
        Z = 1.0+1.0j
        print(Z)
        print(eq(Z,p))
    except:
        print('Crash crash ')
        break
        pass

## get a random complex domain

In [ ]:
N_DEC=4

domain_dict = get_random_domain()
f = zp.get_frame_from_dict(domain_dict)
s = zp.complex_frame_dict_to_string(f, N_DEC)
print(s,'\n%50s'%('(from dictionary)'))
for k, v in domain_dict.items():
    if isinstance(v, complex):
        v_str = zp.complex_to_string(v)
    else:
        v_str = '%i'%(v)
        
    print('%40s: %s'%(k, v_str))

## hash_parameters( ) value for function, parameters and domain - check for duplicates

In [ ]:
fcn_name, eq, p = get_rand_eq_p_set()
domain_dict = get_random_domain()

hash_key = hash_parameters(domain_dict, fcn_name, p)
print(hash_key)

## Produce many pair of scaled images

In [ ]:
test_temporary_dir = '../../test_temporary_dir'
if os.path.isdir(test_temporary_dir) == False:
    os.makedirs(test_temporary_dir)
# os.listdir('../../')
hashy_list = []


small_scale = [128, 128]
large_scale = [255, 255]

n_2_do = 20
for k_do in range(n_2_do):
    fcn_name, eq, p = get_rand_eq_p_set()
    print('\n%3i) %s'%(k_do, fcn_name))
    domain_dict = get_random_domain()

    domain_dict['it_max'] = 64
    domain_dict['max_d'] = 10 / domain_dict['zoom']

    hash_idx = hash_parameters(domain_dict, fcn_name, p)
    if hash_idx in hashy_list:
        print('Impossible, It Cannot Be!, How How How, Who Who Who')
    else:
        hashy_list.append(hash_idx)
        print(hash_idx + '.jpg\n')
        domain_dict['n_rows'] = small_scale[0]
        domain_dict['n_cols'] = small_scale[1]

        domain_dict['dir_path'] = test_temporary_dir

        list_tuple = [(eq, (p))]

        t0 = time.time()
        ET, Z, Z0 = eq_iter.get_primitives(list_tuple, domain_dict)
        I = get_im(ET, Z, Z0)
        print(time.time() - t0, '\t total time')

        display(I)

        domain_dict['n_rows'] = large_scale[0]
        domain_dict['n_cols'] = large_scale[1]

        t0 = time.time()
        ET, Z, Z0 = eq_iter.get_primitives(list_tuple, domain_dict)
        I = get_im(ET, Z, Z0)
        print(time.time() - t0, '\t total time')

        display(I)

        Zd_t, Zr_t, ETn_t = ncp.etg_norm(Z0, Z, ET)
        print('\nZd_t', np.max(Zd_t), np.min(Zd_t))
        print('Zr_t', np.max(Zr_t), np.min(Zr_t))
        print('ETn_t', np.max(ETn_t), np.min(ETn_t))

## file nameing ?

In [ ]:
do_da = now_name(prefi_str=fcn_name, suffi_str='.jpg')
print(do_da)
d0_dat = hashy_list[0] + '.jpg'
print(d0_dat)